In [3]:
import tkinter as tk
import time
from queue import Queue, PriorityQueue
import random

CANVAS_SIZE = 600

class StringTransformerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("String Transformation Solver")

        # Create canvas
        self.canvas = tk.Canvas(self.root, width=CANVAS_SIZE, height=CANVAS_SIZE, bg='white')
        self.canvas.grid(row=0, column=0, columnspan=4)

        # Control buttons for each search algorithm
        self.bfs_button = tk.Button(self.root, text="Solve with BFS", command=lambda: self.solve_string_problem(bfs_string, "BFS"))
        self.bfs_button.grid(row=1, column=0)

        self.dfs_button = tk.Button(self.root, text="Solve with DFS", command=lambda: self.solve_string_problem(dfs_string, "DFS"))
        self.dfs_button.grid(row=1, column=1)

#         self.astar_button = tk.Button(self.root, text="A*", command=lambda: self.solve_string_problem(astar_string, "A*"))
#         self.astar_button.grid(row=1, column=2)

        self.bidirectional_button = tk.Button(self.root, text="Bidirectional Search", command=lambda: self.solve_string_problem(bidirectional_search_string, "Bidirectional"))
        self.bidirectional_button.grid(row=1, column=2)

        self.manhattan_button = tk.Button(self.root, text="A* Manhattan", command=lambda: self.solve_string_problem(astar_manhattan_string, "A* Manhattan"))
        self.manhattan_button.grid(row=2, column=0)

        self.euclidean_button = tk.Button(self.root, text="A* Euclidean", command=lambda: self.solve_string_problem(astar_euclidean_string, "A* Euclidean"))
        self.euclidean_button.grid(row=2, column=1)

        self.ucs_button = tk.Button(self.root, text="Solve with UCS", command=lambda: self.solve_string_problem(ucs_string, "UCS"))
        self.ucs_button.grid(row=2, column=2)

        self.random_button = tk.Button(self.root, text="Randomize Words", command=self.generate_random_words)
        self.random_button.grid(row=3, column=1, columnspan=2)

        # Performance Metrics
        self.metrics_label = tk.Label(self.root, text="", font=("Arial", 12), fg="blue")
        self.metrics_label.grid(row=4, column=0, columnspan=4)

        # Initialize start and end words
        self.valid_words = {"cat", "bat", "bot", "bog", "dog", "cog", "cot", "dot", 
                            "dat", "dag", "bag", "bog", "cot", "log", "lot", "hog"}
        self.generate_random_words()

    def generate_random_words(self):
        """Randomly select start and end words from the valid set."""
        words = list(self.valid_words)
        self.start_word = random.choice(words)
        self.end_word = random.choice(words)
        while self.start_word == self.end_word:
            self.end_word = random.choice(words)
        self.display_string_problem()

    def display_string_problem(self):
        """Display the string transformation problem on the canvas."""
        self.canvas.delete("all")
        self.canvas.create_text(CANVAS_SIZE / 2, CANVAS_SIZE / 3,
                                text=f"Transform '{self.start_word}' to '{self.end_word}'",
                                font=("Arial", 20), fill="black")

    def solve_string_problem(self, algorithm, name):
        """Solve the string transformation problem with the chosen algorithm."""
        start_time = time.perf_counter()
        solution_path, space_used = algorithm(self.start_word, self.end_word, self.valid_words)
        elapsed_time = time.perf_counter() - start_time

        if solution_path:
            self.metrics_label.config(text=f"{name} - Time: {elapsed_time:.6f}s, Space: {space_used} nodes")
            print(f"{name} Solution: {solution_path}")
            self.animate_solution(solution_path)
        else:
            self.metrics_label.config(text=f"{name} could not find a solution.")
            print(f"{name} could not find a solution.")

    def animate_solution(self, solution_path):
        """Animate the solution path smoothly on the GUI."""
        y_position = CANVAS_SIZE / 2
        self.canvas.delete("all")

        for word in solution_path:
            self.canvas.delete("all")
            self.canvas.create_text(CANVAS_SIZE / 2, y_position,
                                    text=word, font=("Arial", 20), fill="green")
            self.root.update()
            time.sleep(1)  # Delay for smooth animation


# BFS Algorithm for String Transformation
def bfs_string(start, end, valid_words):
    queue = Queue()
    queue.put([start])
    visited = set([start])
    max_space = 1

    while not queue.empty():
        max_space = max(max_space, queue.qsize())
        path = queue.get()
        word = path[-1]

        if word == end:
            return path, max_space

        for next_word in valid_words:
            if next_word not in visited and sum(a != b for a, b in zip(word, next_word)) == 1:
                visited.add(next_word)
                queue.put(path + [next_word])

    return None, max_space


# DFS Algorithm for String Transformation
def dfs_string(start, end, valid_words):
    stack = [[start]]
    visited = set([start])
    max_space = 1

    while stack:
        max_space = max(max_space, len(stack))
        path = stack.pop()
        word = path[-1]

        if word == end:
            return path, max_space

        for next_word in valid_words:
            if next_word not in visited and sum(a != b for a, b in zip(word, next_word)) == 1:
                visited.add(next_word)
                stack.append(path + [next_word])

    return None, max_space


# Generalized A* Algorithm
def astar_string_with_heuristic(start, end, valid_words, heuristic):
    pq = PriorityQueue()
    pq.put((0, [start]))
    visited = set([start])
    max_space = 1

    while not pq.empty():
        max_space = max(max_space, pq.qsize())
        _, path = pq.get()
        word = path[-1]

        if word == end:
            return path, max_space

        for next_word in valid_words:
            if next_word not in visited and sum(a != b for a, b in zip(word, next_word)) == 1:
                visited.add(next_word)
                new_path = path + [next_word]
                cost = len(new_path) + heuristic(next_word, end)
                pq.put((cost, new_path))

    return None, max_space


# A* Manhattan Algorithm
def astar_manhattan_string(start, end, valid_words):
    def manhattan_heuristic(word1, word2):
        return sum(a != b for a, b in zip(word1, word2))

    return astar_string_with_heuristic(start, end, valid_words, manhattan_heuristic)


# A* Euclidean Algorithm
def astar_euclidean_string(start, end, valid_words):
    def euclidean_heuristic(word1, word2):
        return (sum(a != b for a, b in zip(word1, word2))) ** 0.5

    return astar_string_with_heuristic(start, end, valid_words, euclidean_heuristic)


# Uniform Cost Search (UCS)
def ucs_string(start, end, valid_words):
    return astar_string_with_heuristic(start, end, valid_words, lambda word1, word2: 0)


# Bidirectional Search for String Transformation
def bidirectional_search_string(start, end, valid_words):
    if start == end:
        return [start], 1

    front_queue = Queue()
    back_queue = Queue()
    front_queue.put([start])
    back_queue.put([end])

    front_visited = {start: [start]}
    back_visited = {end: [end]}
    max_space = 2

    while not front_queue.empty() and not back_queue.empty():
        max_space = max(max_space, front_queue.qsize() + back_queue.qsize())

        front_path = front_queue.get()
        front_word = front_path[-1]

        back_path = back_queue.get()
        back_word = back_path[-1]

        if front_word in back_visited:
            return front_path + back_visited[front_word][::-1][1:], max_space

        if back_word in front_visited:
            return front_visited[back_word] + back_path[::-1][1:], max_space

        for next_word in valid_words:
            if next_word not in front_visited and sum(a != b for a, b in zip(front_word, next_word)) == 1:
                front_visited[next_word] = front_path + [next_word]
                front_queue.put(front_path + [next_word])

            if next_word not in back_visited and sum(a != b for a, b in zip(back_word, next_word)) == 1:
                back_visited[next_word] = back_path + [next_word]
                back_queue.put(back_path + [next_word])

    return None, max_space


# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = StringTransformerApp(root)
    root.mainloop()


BFS Solution: ['hog', 'dog']
DFS Solution: ['bat', 'dat']
Bidirectional Solution: ['cog', 'log', 'dog']
A* Manhattan Solution: ['hog', 'bog', 'bag', 'bat']
A* Euclidean Solution: ['hog', 'log']
UCS Solution: ['cat', 'cot', 'dot']
